In [1]:
import os
from pathlib import Path
from datasets import load_dataset

import nest_asyncio
nest_asyncio.apply()

main_dir = Path("/home/lyb")
data_dir = main_dir / "RAG/data/eli5_data"

In [ ]:
def load_eli5_dataset(save_path):
    # set file path
    file_path = "MarkrAI/eli5_sample_autorag"

    # load dataset
    corpus_dataset = load_dataset(file_path, "corpus")['train'].to_pandas()
    qa_train_dataset = load_dataset(file_path, "qa")['train'].to_pandas()
    qa_test_dataset = load_dataset(file_path, "qa")['test'].to_pandas()

    # save data
    if os.path.exists(os.path.join(save_path, "corpus.parquet")) is True:
        raise ValueError("corpus.parquet already exists")
    if os.path.exists(os.path.join(save_path, "qa.parquet")) is True:
        raise ValueError("qa.parquet already exists")
    corpus_dataset.to_parquet(os.path.join(save_path, "corpus.parquet"))
    qa_train_dataset.to_parquet(os.path.join(save_path, "qa_train.parquet"))
    qa_test_dataset.to_parquet(os.path.join(save_path, "qa_test.parquet"))


load_eli5_dataset(data_dir)

In [5]:
import pandas as pd
qa_df = pd.read_parquet(data_dir / 'qa_train.parquet')
sample_qa_df = qa_df.sample(20, random_state=42) # In this sample code, we will only optimize pipeline with 20 samples just for testing.
sample_qa_df.reset_index(drop=True, inplace=True)
sample_qa_df.to_parquet(data_dir / 'qa_sample.parquet')

from itertools import chain
from autorag.utils.util import to_list
# We drop unused corpus dataframe for faster inference.
corpus_df = pd.read_parquet(data_dir / 'corpus.parquet')
target_retrieval_gts = list(chain.from_iterable(to_list(sample_qa_df["retrieval_gt"].tolist())))
target_retrieval_gts = list(chain.from_iterable(target_retrieval_gts))
sample_corpus_df = corpus_df[corpus_df["doc_id"].isin(target_retrieval_gts)]
sample_corpus_df.reset_index(drop=True, inplace=True)
sample_corpus_df.to_parquet(data_dir / 'corpus_sample.parquet')

[12/20/24 06:10:17] INFO     [__init__.py:100] >> You are using API version of AutoRAG.To use local ]8;id=567134;file:///home/lyb/RAG/AutoRAG/autorag/__init__.py\__init__.py]8;;\:]8;id=746580;file:///home/lyb/RAG/AutoRAG/autorag/__init__.py#100\100]8;;\
                             version, run pip install 'AutoRAG[gpu]'                                               

                    INFO     [__init__.py:128] >> You are using API version of AutoRAG.To use local ]8;id=406636;file:///home/lyb/RAG/AutoRAG/autorag/__init__.py\__init__.py]8;;\:]8;id=402932;file:///home/lyb/RAG/AutoRAG/autorag/__init__.py#128\128]8;;\
                             version, run pip install 'AutoRAG[gpu]'                                               

In [ ]:
import pandas as pd
from autorag.schema.metricinput import MetricInput
from autorag.evaluation import evaluate_generation

# Load QA dataset
qa_df = pd.read_parquet("qa.parquet", engine="pyarrow")

# Prepare MetricInput list
metric_inputs = [
    MetricInput(query=row["query"], generation_gt=row["generation_gt"])
    for _, row in qa_df.iterrows()
]

# Define custom generation function with decorator
@evaluate_generation(
    metric_inputs=metric_inputs,
    metrics=["bleu", "meteor", "rouge"]
)
def custom_generation(queries):
    # Implement your generation logic
    return generated_texts, [[1, 30]] * len(generated_texts), [[-1, -1.3]] * len(generated_texts)

# Evaluate generation performance
generation_result_df = custom_generation(qa_df["query"].tolist())

In [2]:
from autorag.evaluator import Evaluator
evaluator = Evaluator(qa_data_path=(data_dir/'qa_sample.parquet').as_posix(), corpus_data_path=(data_dir/'corpus.parquet').as_posix(),
                      project_dir='./eli5')

/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/pydantic/_internal/_fiel

In [ ]:
evaluator.start_trial('./ollama_config.yaml', skip_validation=True)

[12/22/24 03:13:52] INFO     [evaluator.py:228] >> Embedding BM25 corpus...                        ]8;id=200867;file:///home/lyb/RAG/AutoRAG/autorag/evaluator.py\evaluator.py]8;;\:]8;id=58112;file:///home/lyb/RAG/AutoRAG/autorag/evaluator.py#228\228]8;;\

[12/22/24 03:13:53] INFO     [evaluator.py:248] >> BM25 corpus embedding complete.                 ]8;id=959067;file:///home/lyb/RAG/AutoRAG/autorag/evaluator.py\evaluator.py]8;;\:]8;id=461185;file:///home/lyb/RAG/AutoRAG/autorag/evaluator.py#248\248]8;;\

Output()

                    INFO     [SentenceTransformer.py:218] >> Load pretrained             ]8;id=287052;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=602151;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py#218\218]8;;\
                             SentenceTransformer:                                                                  
                             sentence-transformers/all-mpnet-base-v2                                               

/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: 
FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will 
be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: 
https://github.com/huggingface/transformers/issues/31884
  warnings.warn(

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                    INFO     [base.py:28] >> Deleting retrieval node - VectorDB module...                ]8;id=801493;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py\base.py]8;;\:]8;id=748636;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py#28\28]8;;\

                    INFO     [base.py:18] >> Initialize retrieval node - BM25                            ]8;id=877577;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py\base.py]8;;\:]8;id=202447;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py#18\18]8;;\

[12/22/24 03:14:11] INFO     [base.py:31] >> Running retrieval node - BM25 module...                     ]8;id=295048;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py\base.py]8;;\:]8;id=431566;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py#31\31]8;;\

                    INFO     [base.py:28] >> Deleting retrieval node - BM25 module...                    ]8;id=706177;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py\base.py]8;;\:]8;id=287295;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py#28\28]8;;\

In [7]:
from autorag.deploy import extract_best_config
extract_best_config(trial_path='./eli5/0', output_path='./eli5/0/best.yaml')

{'node_lines': [{'node_line_name': 'retrieve_node_line',
   'nodes': [{'node_type': 'retrieval',
     'strategy': {'metrics': ['retrieval_f1',
       'retrieval_recall',
       'retrieval_precision']},
     'modules': [{'module_type': 'HybridCC',
       'top_k': 3,
       'target_modules': ('VectorDB', 'BM25'),
       'weights': (0.3, 0.7),
       'weight': 0.0,
       'target_module_params': ({'top_k': 3, 'vectordb': 'chroma_mpnet'},
        {'top_k': 3})}]}]},
  {'node_line_name': 'post_retrieve_node_line',
   'nodes': [{'node_type': 'prompt_maker',
     'strategy': {'metrics': ['meteor', 'rouge', 'bert_score']},
     'modules': [{'module_type': 'Fstring',
       'prompt': 'Read the passages and answer the given question. \n Question: {query} \n Passage: {retrieved_contents} \n Answer : '}]},
    {'node_type': 'generator',
     'strategy': {'metrics': ['meteor', 'rouge', 'bert_score']},
     'modules': [{'module_type': 'LlamaIndexLLM',
       'llm': 'ollama',
       'model': 'llama3'

In [8]:
from autorag.deploy import Runner
runner = Runner.from_yaml('./eli5/0/best.yaml', project_dir='./eli5')
runner.run('who are you?')

[12/22/24 03:39:31] INFO     [base.py:18] >> Initialize retrieval node - HybridCC                        ]8;id=808566;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py\base.py]8;;\:]8;id=605026;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py#18\18]8;;\

                    INFO     [base.py:18] >> Initialize retrieval node - VectorDB                        ]8;id=159339;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py\base.py]8;;\:]8;id=567816;file:///home/lyb/RAG/AutoRAG/autorag/nodes/retrieval/base.py#18\18]8;;\

[12/22/24 03:39:32] INFO     [SentenceTransformer.py:218] >> Load pretrained             ]8;id=301388;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=192735;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py#218\218]8;;\
                             SentenceTransformer:                                                                  
                             sentence-transformers/all-mpnet-base-v2                                               

/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[12/22/24 03:39:35] INFO     [SentenceTransformer.py:357] >> 2 prompts are loaded, with  ]8;id=751678;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=263505;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py#357\357]8;;\
                             the keys: ['query', 'text']                                                           

...

                    INFO     [base.py:23] >> Running prompt maker node - Fstring module...               ]8;id=705491;file:///home/lyb/RAG/AutoRAG/autorag/nodes/promptmaker/base.py\base.py]8;;\:]8;id=229316;file:///home/lyb/RAG/AutoRAG/autorag/nodes/promptmaker/base.py#23\23]8;;\

                    INFO     [base.py:26] >> Running generator node - LlamaIndexLLM module...            ]8;id=115989;file:///home/lyb/RAG/AutoRAG/autorag/nodes/generator/base.py\base.py]8;;\:]8;id=705149;file:///home/lyb/RAG/AutoRAG/autorag/nodes/generator/base.py#26\26]8;;\

[12/22/24 03:39:43] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=614998;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=50014;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             http://localhost:11434/api/chat "HTTP/1.1 200 OK"                                     

/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


'Based on the passages provided, it seems that there is no clear answer to the question "Who are you?" as the passages appear to be unrelated and do not provide any information about the person\'s identity or characteristics.\n\nHowever, if I had to make an educated guess based on the tone and style of the passages, I would say that the author appears to be someone who values honesty, openness, and creativity. The author seems to be comfortable sharing their thoughts and feelings with others, and is not afraid to express themselves freely.'

{'node_lines': [{'node_line_name': 'retrieve_node_line',
   'nodes': [{'modules': [{'module_type': 'HybridCC',
       'target_module_params': [{'top_k': 3, 'vectordb': 'chroma_mpnet'},
        {'top_k': 3}],
       'target_modules': ['VectorDB', 'BM25'],
       'top_k': 3,
       'weight': 0.0,
       'weights': [0.3, 0.7]}],
     'node_type': 'retrieval',
     'strategy': {'metrics': ['retrieval_f1',
       'retrieval_recall',
       'retrieval_precision']}}]},
  {'node_line_name': 'post_retrieve_node_line',
   'nodes': [{'modules': [{'module_type': 'Fstring',
       'prompt': 'Read the passages and answer the given question. \n Question: {query} \n Passage: {retrieved_contents} \n Answer : '}],
     'node_type': 'prompt_maker',
     'strategy': {'metrics': ['meteor', 'rouge', 'bert_score']}},
    {'modules': [{'batch': 1,
       'llm': 'ollama',
       'model': 'llama3',
       'module_type': 'LlamaIndexLLM',
       'temperature': 0.5}],
     'node_type': 'generator',
     'strategy'

In [5]:
!autorag run_web --yaml_path ./eli5/0/best.yaml --project_dir ./eli5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[12/21/24 16:19:56] INFO     [config.py:58] >> PyTorch version      ]8;id=239230;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/datasets/config.py\config.py]8;;\:]8;id=803982;file:///home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/datasets/config.py#58\58]8;;\
                             2.5.1 available.                                   
/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/lyb/.conda/envs/autorag/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/lyb/.c